In [1]:
import pandas as pd

data = pd.read_csv('Crop_Dataset.csv')

from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, train_size=0.9, random_state=42)

train_data.shape, test_data.shape

((1980, 12), (220, 12))

In [2]:
n_classes = len(data['Label_Encoded'].unique())
n_classes

22

In [3]:
import torch

device = (
    "mps" if torch.cuda.is_available() else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [4]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, data):
        self.inputs = data.iloc[:, 0:-2].values.astype('float32')
        self.outputs = data.iloc[:, -1].values.astype('int')

    def __getitem__(self, index):
        return self.inputs[index], self.outputs[index]

    def __len__(self):
        return len(self.inputs)

In [5]:
batch_size = 64

train_data_loader = torch.utils.data.DataLoader(CustomDataset(train_data), batch_size=batch_size, shuffle=True)
test_data_loader = torch.utils.data.DataLoader(CustomDataset(test_data), batch_size=batch_size, shuffle=True)

In [6]:
import torch.nn as nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(10, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 22),
            nn.Sigmoid()
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [7]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=10, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=22, bias=True)
    (5): Sigmoid()
  )
)


In [8]:
# testing predictions

_X = torch.rand(1, 10, device=device)
logits = model(_X)
predicted_prob = nn.Softmax(dim=1)(logits)
y_pred = predicted_prob.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([19])


In [9]:
learning_rate = 1e-3
batch_size = 64
epochs = 50

loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [10]:
import wandb

configs = {
    "learning_rate": learning_rate,
    "architecture": "ANN",
    "dataset": "Crop",
    "epochs": epochs,
    "batch_size": batch_size
}

wandb.init(
    project="Intellihack_1",
    config=configs
)

wandb: Currently logged in as: lakshith. Use `wandb login --relogin` to force relogin


In [11]:
def train_loop(dataloader):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        loss, current = loss.item(), batch * batch_size + len(X)
        
        wandb.log({
            "Loss": loss
        })
        print(f"\rloss: {loss:>7f}  [{current:>5d}/{size:>5d}]", end="")

In [12]:
def test_loop(dataloader):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    
    print({"test_loss": test_loss, "test_acc": correct})

In [13]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    print("")
    train_loop(train_data_loader)
    test_loop(test_data_loader)

Epoch 1
-------------------------------

loss: 3.192411  [ 1980/ 1980]{'test_loss': 3.176541745662689, 'test_acc': 0.05454545454545454}
Epoch 2
-------------------------------

loss: 3.201802  [ 1980/ 1980]{'test_loss': 3.1726194024086, 'test_acc': 0.05454545454545454}
Epoch 3
-------------------------------

loss: 3.167632  [ 1980/ 1980]{'test_loss': 3.1824839115142822, 'test_acc': 0.05454545454545454}
Epoch 4
-------------------------------

loss: 3.093004  [ 1980/ 1980]{'test_loss': 3.131054401397705, 'test_acc': 0.05454545454545454}
Epoch 5
-------------------------------

loss: 3.156684  [ 1980/ 1980]{'test_loss': 3.13827246427536, 'test_acc': 0.05454545454545454}
Epoch 6
-------------------------------

loss: 3.160964  [ 1980/ 1980]{'test_loss': 3.1212083101272583, 'test_acc': 0.05454545454545454}
Epoch 7
-------------------------------

loss: 3.175401  [ 1980/ 1980]{'test_loss': 3.130158305168152, 'test_acc': 0.05454545454545454}
Epoch 8
-------------------------------

loss: 3.